## LHE reader

In [1]:
from __future__ import print_function

In [2]:
import uproot
from uproot_methods import TLorentzVectorArray, TLorentzVector
import numpy as np
import re
from itertools import combinations
from pprint import pprint

In [3]:
events = {'part': [], 'clus': []}
with open('cmsgrid_final.lhe') as f:
    lines = f.readlines()
    nlines, i = len(lines), 0
    while i < nlines:
        l = lines[i].strip()
        if l=='<event>':
            ## enter the event block
            npart = int(lines[i+1].split()[0])
            part_info = []
            for j in range(npart):
                ltmp = lines[i+j+2].split()
                part_info.append(tuple([int(ltmp[0]), int(ltmp[1])] + [float(ltmp[k]) for k in range(6,11)]))
            events['part'].append(part_info)
            i += npart
        if l=='<clustering>':
            clus_info, iclus = [], 0
            while lines[i+iclus+1][:5]=='<clus':
                ltmp = re.findall('[a-z0-9\.\-]+', lines[i+iclus+1])
                clus_info.append(tuple([float(ltmp[2])] + [int(ltmp[k]) for k in range(3,7)]))
                iclus += 1
            events['clus'].append(clus_info)
            i += iclus
        i += 1

In [4]:
ie=3
part_info = np.array(events['part'][ie])
p4 = list(TLorentzVectorArray.from_cartesian(part_info[:,2], part_info[:,3], part_info[:,4], part_info[:,5]))
iterlist = list(range(2, len(p4)))
for i in iterlist:
    if part_info[i,1] == 2: # statu=2 => intermidate particle
        iterlist.remove(i)
istep = 1
while len(iterlist) > 0:
    print('--- Start iteration', istep, ', particle list:', iterlist, ' ---\n')
    ktpool = {}
    for i in iterlist:
#         ktpool[i] = np.sqrt(p4[i].energy**2 - p4[i].z**2)
        ktpool[i] = p4[i].pt
    for i, j in combinations(iterlist, 2):
        ktpool[(i,j)] = np.sqrt(max(p4[i].mag2, p4[j].mag2) + min(p4[i].pt2, p4[j].pt2) * p4[i].delta_r(p4[j]))
#         p4ij = p4[i]+p4[j]; ktpool[(i,j)] = np.sqrt(p4ij.energy**2 - p4ij.z**2)
#         p4ij = p4[i]+p4[j]; ktpool[(i,j)] = p4ij.mass
    
    pprint(ktpool)

    minkey = min(ktpool, key=ktpool.get)
    if isinstance(minkey, int):
        iterlist.remove(minkey)
    else:
        p4[minkey[0]] += p4[minkey[1]]
        iterlist.remove(minkey[1]) # remove j while keep i
    print('find minimum Durham kT at index:', minkey, ',  value =', ktpool[minkey], '\n')
    istep += 1

--- Start iteration 1 , particle list: [3, 4, 5]  ---

{3: 439.4119777027959,
 4: 7.620114796312987,
 5: 438.16433708218017,
 (3, 4): 10.235070036786762,
 (3, 5): 776.6164049831569,
 (4, 5): 9.273753292535666}
find minimum Durham kT at index: 4 ,  value = 7.62011479631 

--- Start iteration 2 , particle list: [3, 5]  ---

{3: 439.4119777027959, 5: 438.16433708218017, (3, 5): 776.6164049831569}
find minimum Durham kT at index: 5 ,  value = 438.164337082 

--- Start iteration 3 , particle list: [3]  ---

{3: 439.4119777027959}
find minimum Durham kT at index: 3 ,  value = 439.411977703 



In [5]:
events['clus'][ie]

[(90.012, 4, 5, -1, -1), (438.165, 4, 6, -1, -1), (897.144, 2, 4, 1, -1)]